In [ ]:
%pip install openai num2words matplotlib plotly scipy scikit-learn pandas tiktoken redis langchain

In [2]:
# import openai
# import re
# import requests
# import sys
# from num2words import num2words
import os
import pandas as pd
# import numpy as np
# from openai.embeddings_utils import get_embedding
# import tiktoken
# from typing import List
# from langchain.embeddings import OpenAIEmbeddings
# from langchain.vectorstores.redis import Redis as RedisVectorStore

In [30]:
# Read and filter data

df=pd.read_csv(os.path.join(os.getcwd(),'US_Accidents_March23.csv'), usecols=['Start_Time', 'Description', 'City'])
df

,Start_Time,Description,City
0,2016-02-08 05:46:00,Right lane blocked due to accident on I-70 Eas...,Dayton
1,2016-02-08 06:07:59,Accident on Brice Rd at Tussing Rd. Expect del...,Reynoldsburg
2,2016-02-08 06:49:27,Accident on OH-32 State Route 32 Westbound at ...,Williamsburg
3,2016-02-08 07:23:34,Accident on I-75 Southbound at Exits 52 52B US...,Dayton
4,2016-02-08 07:39:07,Accident on McEwen Rd at OH-725 Miamisburg Cen...,Dayton
...,...,...,...
7728389,2019-08-23 18:03:25,At Market St - Accident.,Riverside
7728390,2019-08-23 19:11:30,At Camino Del Rio/Mission Center Rd - Accident.,San Diego
7728391,2019-08-23 19:00:21,At Glassell St/Grand Ave - Accident. in the ri...,Orange
7728392,2019-08-23 19:00:21,At CA-90/Marina Fwy/Jefferson Blvd - Accident.,Culver City


In [31]:
# Clean data

df['Start_Time'] = pd.to_datetime(df['Start_Time'], format='mixed')
df['Start_Time'] = df['Start_Time'].dt.strftime('%Y-%m-%d')
df['Description'] = df['Description'].str.rstrip('.')
df

'Right lane blocked due to accident on I-70 Eastbound at Exit 41 OH-235 State Route 4'

In [32]:
# Merge data

df['merged'] = df.apply(lambda x: x.astype(str)['Description'] + ' on ' + x.astype(str)['Start_Time'] + ' in the city of ' + x.astype(str)['City'], axis=1)
df['merged'].iloc[0]

'Right lane blocked due to accident on I-70 Eastbound at Exit 41 OH-235 State Route 4 on 2016-02-08 in the city of Dayton'

In [ ]:
#Check how many tokens it will require to encode all the accidents

tokenizer = tiktoken.get_encoding("cl100k_base")
df['n_tokens'] = df["Plot"].apply(lambda x: len(tokenizer.encode(x)))
df = df[df.n_tokens<8192]
print('Number of accidents: ' + str(len(df))) # print number of accidents remaining in dataset
print('Number of tokens required:' + str(df['n_tokens'].sum())) # print number of tokens

In [ ]:
# load everything into Langchain

from langchain.document_loaders import DataFrameLoader
loader = DataFrameLoader(df, page_content_column="Plot" )
movie_list = loader.load()

Convert this notebook to Python app `jupyter nbconvert --to python rag-demo.ipynb`